In [1]:
import pandas as pd
train_file = "dahta/train .csv"
mapping_file = "data/misconception_mapping.csv"

In [ ]:
train_df = pd.read_csv(train_file)
mapping_df = pd.read_csv(mapping_file)
print("Train Data Preview:")
print(train_df.head())
print("\nMapping Data Preview:")
print(mapping_df.head())

### Map the correct answer

In [ ]:
# Function to map CorrectAnswer to corresponding AnswerText
def map_correct_answer(row):
    # Map the correct answer to the corresponding column value
    answer_column = f"Answer{row['CorrectAnswer']}Text"
    return row[answer_column]

# Apply the function to the DataFrame
train_df = pd.read_csv(train_file)
train_df['CorrectAnswerText'] = train_df.apply(map_correct_answer, axis=1)

# Display the result
train_df[['CorrectAnswer', 'CorrectAnswerText']]


In [4]:
def map_misconception(misconception_id, mapping):
    if pd.notna(misconception_id):
        return mapping.get(misconception_id, "Unknown")
    return None

### Map the misconception names

In [5]:
# Create a dictionary to map misconception IDs to names
misconception_dict = mapping_df.set_index('MisconceptionId')['MisconceptionName'].to_dict()
train_df['MisconceptionA'] = train_df['MisconceptionAId'].apply(map_misconception, args=(misconception_dict,))
train_df['MisconceptionB'] = train_df['MisconceptionBId'].apply(map_misconception, args=(misconception_dict,))
train_df['MisconceptionC'] = train_df['MisconceptionCId'].apply(map_misconception, args=(misconception_dict,))
train_df['MisconceptionD'] = train_df['MisconceptionDId'].apply(map_misconception, args=(misconception_dict,))


In [ ]:
print("\nMapped Misconception Names:")
print(train_df[['MisconceptionA', 'MisconceptionB', 'MisconceptionC', 'MisconceptionD']].head())

In [ ]:
# Split the dataframe into separate rows for each answer
split_rows = []
for _, row in train_df.iterrows():
    split_rows.append({
        'QuestionId_Answer': f"{row['QuestionId']}_A",
        'QuestionId': row['QuestionId'],
        'ConstructId': row['ConstructId'],
        'ConstructName': row['ConstructName'],
        'CorrectAnswer': row['CorrectAnswer'],
        'SubjectId': row['SubjectId'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'AnswerText': row['AnswerAText'],
        'MisconceptionId': row['MisconceptionAId'],
        'MisconceptionName': row['MisconceptionA'],
        'CorrectAnswerText': row['CorrectAnswerText']

    })
    split_rows.append({
        'QuestionId_Answer': f"{row['QuestionId']}_B",
        'QuestionId': row['QuestionId'],
        'ConstructId': row['ConstructId'],
        'ConstructName': row['ConstructName'],
        'CorrectAnswer': row['CorrectAnswer'],
        'SubjectId': row['SubjectId'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'AnswerText': row['AnswerBText'],
        'MisconceptionId': row['MisconceptionBId'],
        'MisconceptionName': row['MisconceptionB'],
        'CorrectAnswerText': row['CorrectAnswerText']
    })
    split_rows.append({
        'QuestionId_Answer': f"{row['QuestionId']}_C",
        'QuestionId': row['QuestionId'],
        'ConstructId': row['ConstructId'],
        'ConstructName': row['ConstructName'],
        'CorrectAnswer': row['CorrectAnswer'],
        'SubjectId': row['SubjectId'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'AnswerText': row['AnswerCText'],
        'MisconceptionId': row['MisconceptionCId'],
        'MisconceptionName': row['MisconceptionC'],
        'CorrectAnswerText': row['CorrectAnswerText']
    })
    split_rows.append({
        'QuestionId_Answer': f"{row['QuestionId']}_D",
        'QuestionId': row['QuestionId'],
        'ConstructId': row['ConstructId'],
        'ConstructName': row['ConstructName'],
        'CorrectAnswer': row['CorrectAnswer'],
        'SubjectId': row['SubjectId'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'AnswerText': row['AnswerDText'],
        'MisconceptionId': row['MisconceptionDId'],
        'MisconceptionName': row['MisconceptionD'],
        'CorrectAnswerText': row['CorrectAnswerText']
    })

split_df = pd.DataFrame(split_rows)
split_df = split_df.dropna(subset=['MisconceptionName'])
print("\nSplit Data Preview:")
print(split_df.head())


In [ ]:
for index, item in split_df.iterrows():
    print(f"QuestionText: {item['QuestionText']}, ConstructName: {item['ConstructName']},CorrectAnswerText: {item['CorrectAnswerText']}, AnswerText: {item['AnswerText']}")


In [ ]:
split_df

### Generate the prompt

In [ ]:
from prompt import math_problems_anlysis_prompt

# Generate the prompt for each row in the DataFrame
def generate_prompt(row):
    return math_problems_anlysis_prompt.format(
        question=row['QuestionText'], 
        construct=row['ConstructName'], 
        goodanswer=row['CorrectAnswerText'], 
        badanswer=row['AnswerText']
    )

prompts = split_df.apply(generate_prompt, axis=1)

# Print the prompts
print("\nPrompts:")
prompts.head()


In [ ]:
len(prompts)

In [12]:
prompts = prompts.reset_index(drop=True)

In [ ]:
prompts[1]

In [ ]:
# from service.llms_service import LLM_vllm_Service 
# model_path = 'models/raw_model/mistral'
# tensor_parallel_size = 2
# llm_service = LLM_vllm_Service(model_path, tensor_parallel_size)
# question = [prompts[1]]
# answer = llm_service.generate(question, llm_service.sampling_params)
# print(answer)